<a href="https://colab.research.google.com/github/FariusGitHub/DataScience/blob/master/SOUP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import bs4
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
from string import punctuation
from nltk.stem import PorterStemmer
from datetime import date

# Creating Table based on Eric request to follow below sequence (case sensitive):
# Title, Company, Location, Summary, Description, Date, Duration, Link

# First iteration & process for Title, Company, Location, Date, Duration, Link
# Second iteration for Description
# Third iteration for Summary

soup = bs4.BeautifulSoup(requests.get(\
'https://www.monster.ca/jobs/search/?q='
                'data-scientist'
                '__2C-data-engineer'
                '__2C-machine-learning-engineer'
                '__2C-data-analyst'
                '&where=Canada&intcid=skr_navigation_nhpso_searchMain&page=10'\
                                     ).text, 'lxml')

#inserting Title, Company, Location, Date, Duration column : FIRST iteration
Company=[];Location=[];Title=[];Duration=[]; Link=[]; 
for i,j in enumerate(tqdm(soup.select('.company'))):
            Company  += [j.text.strip('\n')]
            Location += [soup.select('.location')[i+1].text.strip('\n\r')]
            Title    += [soup.select('.title')[i+3].text.strip('\r\n')]   
            Duration   += ['30 days or more' if soup.find_all(class_=\
                        'meta flex-col')[i].text[1:3]=='30' else\
                        'posted today'    if soup.find_all(class_=\
                        'meta flex-col')[i].text[1:3].isalpha() else\
                                            soup.find_all(class_=\
                        'meta flex-col')[i].text[1:3] + ' days ago']
            Link  += [soup.select('.title')[i+3].a.get('href')]

monster = pd.DataFrame(np.asarray(list(zip(Title, Company, Location, \
          Duration, Link))), columns =['Title', 'Company', 'Location',\
          'Duration', 'Link']).drop_duplicates()\
          .reset_index(drop=True)

#inserting 'Date' column : today's date when this code run, no iteration needed
monster.insert(3, 'Date', date.today())

#inserting 'Description' column : SECOND iteration 
desc=[]
for i in tqdm(monster['Link']):
    try:
        desc += [bs4.BeautifulSoup(requests.get(i).text,\
                'lxml').select('.row')[3].text]
    except:
        desc += ['error during downloading']
monster.insert(3, 'Description', desc) 

#Inserting 'Summary' column : THIRD iteration          
jargons=pd.read_csv("https://farius.s3.amazonaws.com/jargons_01.csv",\
                    header=None)
ps = PorterStemmer()

Summary={};ngram1=[]; ngram2=[]; ngram3=[]
for k,j in enumerate(tqdm(desc)):
    cleaned = ''.join([char if char not in punctuation else ' '\
              for char in j.lower() ])
    ngram1=set([word for word in [cleaned.split()[i] \
              for i in range(len(cleaned.split()))] \
              if word in jargons[0].values.tolist()])
    ngram2=set([word for word in [cleaned.split()[i] +' '+ \
              cleaned.split()[i + 1] for i in range(len(cleaned.split()) - 1)] \
              if word in jargons[0].values.tolist()])
    ngram3=set([word for word in [cleaned.split()[i] +' '+ \
              cleaned.split()[i + 1] +' '+ ps.stem(cleaned.split()[i + 2]) \
              for i in range(len(cleaned.split()) - 2)] \
              if word in jargons[0].values.tolist()])
    Summary[k]=list(ngram1)+list(ngram2)+list(ngram3)
    
monster.insert(3, 'Summary', Summary.values())

monster.style.hide_index()

# def make_clickable(val):
#     return '<a target="_blank" href="{}">{}</a>'.format(val, val)

# monster.style.format({'Link': make_clickable})

100%|██████████| 169/169 [00:16<00:00, 10.20it/s]
